This notebook scrapes the info data for each driver (e.g. team color)

In [2]:
import fastf1 as ff1
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
all_gps = []
for year in range(2020,2025):
    temp_df = pd.DataFrame(ff1.get_event_schedule(year)["RoundNumber"])
    temp_df["year"] = year
    if year != 2024:
        all_gps.append(temp_df)
    else:
        all_gps.append(temp_df.loc[:8])




df_all_gps = pd.concat(all_gps)

req         WARNING 	DEFAULT CACHE ENABLED! (10.79 GB) /Users/max/Library/Caches/fastf1


In [4]:
df_all_gps = df_all_gps.replace({0:None})
df_all_gps = df_all_gps.dropna()
df_all_gps = df_all_gps.reset_index(drop=True)

In [24]:
all_driver_data = []
for _,gp_data in df_all_gps.iterrows():
    year = gp_data["year"]
    gp = gp_data["RoundNumber"]

    session = ff1.get_session(year,gp,"R")
    session.load()

    for driver in session.drivers:
        
        try:
            driver_info = pd.DataFrame(session.get_driver(driver)).transpose()
            driver_info.insert(0,"year",year)
            driver_info.insert(0,"round_number",gp)
            all_driver_data.append(driver_info)
        
        except:
            print("No data available")
        # add the first row to end to make the lap complete
        


df_all_driver_data = pd.concat(all_driver_data)



core           INFO 	Loading data for Austrian Grand Prix - Race [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver 77: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 16: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver  4: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 44: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 55: Lap timing integrity check failed for 1 lap(s)
core        WARNING 	Driver 11: Lap timing integrity check failed for 1 

In [27]:
df_all_driver_data.to_parquet("all_driver_data.parquet",index=False)
